# High Speed E-Stop CCW/Rotator Integration Test

This notebook performs a high speed E-Stop interlock scenario integration test between the Camera Cable Wrap (CCW) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [ ]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
test_message = "E-Stop MTPtg_Rotator_CCW Integration Test"

In [ ]:
d = salobj.Domain()

In [ ]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [ ]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

# Move to 0 deg Starting Position

In [ ]:
await asyncio.sleep(0.1)

print("Move to 0.0 deg for starting position")

await rot.cmd_positionSet.set_start(angle=0.0,timeout=10.)

await rot.cmd_move.start(timeout=10.)

# Send Track Command to +90 deg

In [ ]:
print("Azimuth = 0 and rot_tel= 89 deg")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)
await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

# For Testing - This is pushing E-Stop

In [ ]:
await asyncio.sleep(20)
await mtptg.cmd_stopTracking.start(timeout=30)
await rot.cmd_stop.start(timeout=30)

In [ ]:
script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"END - {test_message}")